# Validación de soluciones de modelos con polítropa Maestra
## Variando el índice polítropo $n$ y $\sigma \ (= \frac{P_{c}}{\rho_{c}})$

Este cuaderno muestra la comparación entre los radios y masas adimensionales en el borde ($\xi_{b},\eta_b$) para la EoS polítropa maestra con $\alpha = \beta = 0$, contra los radios y masas adimensionales ($\bar{\xi}_b, \bar{\eta}_b$) obtenidos en [T. Dallas y V. Geroyannis](http://adsabs.harvard.edu/pdf/1993Ap%26SS.201..249D).

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import integrate
from scipy.integrate import solve_ivp

In [2]:
plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

# Modelado de objetos compactos

## Ecuaciones de estructura

Las ecuaciones de estructura son el resultado de manipular las ecuaciones de campo de Einstein para una métrica y un tensor de energía dado, y son las ecuaciones que hay que resolver para modelar objetos compactos. En el caso de configuraciones estáticas con simetría esférica y anisotropía en la presión, las ecuaciones de estructura consisten en

\begin{eqnarray}
P^{\prime} &=& - \left(\rho + P \right) \frac{m + 4 \pi r^{3} P}{r(r - 2m)} + \frac{2}{r} \left(P_{\perp} - P \right)  \label{EqHid} \tag{1} \\
m^{\prime} &=& 4 \pi r^{2} \rho \label{MasDif} \tag{2}
\end{eqnarray}
donde $\rho$ es densidad de energía, $m$ es masa, $P$ es presión radial, $P_{\perp}$ es presión tangencial y la prima denota derivada respecto a $r$.

## Ecuación de Estado

Una ecuación de estado (EoS por sus siglas en inglés) es un modelo matemático, entre variables de estado, que describe los procesos físicos más importantes que ocurren en un sistema termodinámico.

La EoS polítropa maestra consiste en una relación tal que
\begin{equation}
P = \kappa \rho^{1+\frac{1}{n}} + \alpha \rho - \beta \,. \label{PoliMaestra} \tag{3}
\end{equation}

Por otra parte, se supone que la anisotropía en a presión es de la forma
\begin{equation}
\Delta\equiv P_{\perp} - P = C r (\rho + P) \frac{m + 4 \pi r^3 P}{r(r-2m)} \,, \label{Anisotropia} \tag{4}
\end{equation}
tal que la ecuación (1) queda como
\begin{equation}
\frac{\mathrm{d}P}{\mathrm{d}r} = - h \frac{(\rho + P)(m + 4 \pi  r^3 P)}{r(r-2m)} \,, \label{EqHidCos} \tag{5}
\end{equation}
donde $h = 1 - 2C$, y $C$ cuantifica la anisotropía en el modelo.

## Ecuación de Lane-Emden

Las ecuaciones de estructura pueden escribirse de forma adimensional cuando son dotadas de EoS polítropas. El resultado es conocido como ecuación de Lane-Emden, dada por el cambio de variables
\begin{equation}
\Psi^{n}(\xi) = \frac{\rho}{\rho_{c}} \ , \ \ \eta \left(\xi \right) = \frac{m}{4 \pi \rho_c a^{3}} \quad \textrm{y} \quad r = a\xi \,,
\end{equation}
donde
\begin{equation}
a^{2} = \frac{\Upsilon \left(n + 1 \right)}{4 \pi \rho_c} \ , \ \ \Upsilon = \kappa \rho_{c}^{1/n} = \frac{\sigma - \alpha \left(1 - \varkappa \right)}{1 - \varkappa^{1 + \frac{1}{n}}} \ , \ \ \sigma = \frac{P_{c}}{\rho_{c}} \quad \textrm{y} \quad \varkappa = \frac{\rho_{b}}{\rho_{c}} \,.
\end{equation}
Los subíndices $c$ y $b$ indican que la variable es evaluada en el centro y borde de la configuración, respectivamente.

De esta manera, las EoS $\eqref{PoliMaestra}$ y $\eqref{Anisotropia}$ adimensionales son
\begin{eqnarray}
P &=& \rho_c \left\{\Upsilon \left( \Psi^{n+1} - \varkappa^{1 + \frac{1}{n}} \right) + \alpha \left(\Psi^{n} - \varkappa \right)\right\}  = \rho_c \mathcal{P} \quad \textrm{y} \label{PAdi} \tag{6} \\
\Delta &=& \frac{C \Upsilon (n+1) \left(\eta + \xi^{3} \mathcal{P} \right) \left(\Psi^{n} + \mathcal{P} \right) \rho_{c}}{\xi - 2  \Upsilon \left( n+1 \right) \eta} \,, \label{AniAdi} \tag{7}
\end{eqnarray}
respectivamente.


Finalmente, las ecuaciones $\eqref{EqHidCos}$ y $\eqref{MasDif}$, escritas en su forma adimensional, son
\begin{eqnarray}
\dot{\Psi} &=& - \frac{h \left(\eta + \xi^{3} \mathcal{P}\right) \left(1 + \frac{\mathcal{P}}{\Psi^{n}}\right)}{\xi \left\{\xi-2\,\Upsilon\,\left( n+1 \right) \eta\right\} \left\{1 + \frac{\alpha n}{\Upsilon \left(n+1\right) \Psi}\right\}}  \qquad \textrm{y} \label{PsiPunto} \tag{8} \\
\dot{\eta} &=& \xi^{2}\Psi^{n} \,, \label{EtaPunto} \tag{9}
\end{eqnarray}
donde el punto indica derivada respecto de $\xi$.

Por lo tanto, el sistema de ecuaciones a integrar numéricamente está dado por $\eqref{PsiPunto}$ y $\eqref{EtaPunto}$, con condiciones iniciales
\begin{equation}
\Psi (\xi = 0) = \Psi_{c} = 1 \,, \quad \eta (\xi = 0) = \eta_{c} = 0  \,,
\end{equation}
y con condición de frontera
\begin{equation}
\Psi (\xi = \xi_{b}) = 0 \,.
\end{equation}

In [3]:
# Definiendo sistema de ecuaciones: derivada de Psi y derivada de Eta en función de xi
def funciones(xi,y, alpha, n, h, Upsilon, varkappa):
    psi_ , eta_ = y
    dydxi = [-h*(eta_ + xi**(3)*(Upsilon*(psi_**(n+1)-varkappa**(1+1/n)) + alpha*(psi_**(n) - varkappa)))*(1 + Upsilon*(psi_ - (varkappa**(1+1/n)/psi_**(n))) + alpha*(1 - (varkappa/psi_**(n))))/xi/(xi - 2*Upsilon*(n+1)*eta_) /(1 + alpha*n/Upsilon/(n+1)/psi_) 
            ,xi**(2)*psi_**(n)] 
    return dydxi

In [4]:
c = 2.997*10**(8)          # Velocidad de la luz en unidades S.I.
G = 6.673*10**(-11)        # Constante de gravitación universal en unidades S.I.
Msun = 1.989*10**(30)      # 1 Masa solar en kilogramos

In [5]:
# Se definen los parámetros (de entrada) que caracterizan a cada modelo: n, C, alpha, varkappa, sigma

Lista_n = [0.5,0.099999,1.5,2.0,2.5,3.0,3.5,4.0]        # Lista de índices polítropos n
print('Valores de n: ',end='')                                                    
print(*Lista_n, sep=', ')

C = 0.0  # Valor de C para anisotropía

h = 1 - 2*C

alpha = 0.0 # Constante del término lineal

varkappa = 0.0 # Razón entre densidad en el borde y densidad central

# Sigma: Presión central entre densidad central
Lista_sigma = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
print('Valores de \u03c3: ',end='')                                                    
print(*Lista_sigma, sep=', ')

Valores de n: 0.5, 0.099999, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0
Valores de σ: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9


In [6]:
Listaxi1 = []    # Contenedor de los valores para xi en el borde
ListaEta1 = []   # Contendor de los valores de eta en el borde

In [7]:
%%time

for i in range(len(Lista_n)):
    
    n = Lista_n[i]
    
    for j in range(len(Lista_sigma)):
        
        sigma = Lista_sigma[j]
        
       # Cálculo de Upsilon
        Upsilon = (sigma - alpha*(1 - varkappa))/(1 - varkappa**(1 + 1/n)) 
        
        # Condiciones iniciales
        Psi0 = 1.0
        Eta0 = 0.0
        y0 = [Psi0,Eta0]
        
        # Intervalo de integración
        xi0 = 10**(-15) # Inicio
        ximax = 10**(11)    # Fin, en caso de que la condición para detener la integración no se cumpla
        xi_span = (xi0,ximax) # Espacio de integración
    
        # Condición para detener la integración (Presión adimensional menor que 10**(-15))
        def stop_condition(xi,y,alpha, n, h, Upsilon, varkappa):
            return (1/sigma)*(Upsilon*(y[0]**(n+1) - varkappa**(1 + 1/n)) + alpha*(y[0]**(n) - varkappa)) - 10**-15
        stop_condition.terminal = True
    
        # Solución del sistema de ecuaciones utilizando la rutina "solve_ivp" por medio del método RK45
        # solve_ivp(Ecuaciones a integrar, Espacio de integración, Condiciones iniciales, Método de integración,
        #           Condición para detener la integración)
        soluciones = solve_ivp(funciones,xi_span,y0,method='RK45',events=stop_condition,
                               args=(alpha, n, h, Upsilon, varkappa),max_step=1/50)
        xi = soluciones.t
        Psi = soluciones.y[0]
        Eta = soluciones.y[1]
        
        Listaxi1.append(np.around(xi[-1],7))
        ListaEta1.append(np.around(Eta[-1],7))

C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


Wall time: 18min 52s


In [8]:
pd.options.display.float_format = "{:,.7f}".format

# Lectura de datos tomados de  T. Dallas and V. Geroyannis, “The boundary conditions for relativistic polytropic fluid spheres,
# ”Astrophysics and space science, vol. 201, no. 2, pp. 249–271,1993.
data = pd.read_csv('Validación - Datos Dallas y Geroyannis (1993)',names = ['N', 'Sigma', 'Xi_b', 'Eta_b'],
                  engine='python')

In [9]:
data['Xib'] = Listaxi1      # Añade las soluciones obtenidas, para xi en el borde, a la tabla
data['Etab'] = ListaEta1    # Añade las soluciones obtenidas, para eta en el borde, a la tabla

errorXi = (abs(data['Xi_b'] - data['Xib'])/data['Xi_b'])*100      # Cálculo del error porcentual para xi en el borde
errorEta = (abs(data['Eta_b'] - data['Etab'])/data['Eta_b'])*100  # Cálculo del erro porcentual para eta en el borde
data['Error Xi_b'] = errorXi            # Añade el error, para xi en el borde, a la tabla
data['Error Eta_b'] = errorEta          # Añade el error, para eta en el borde, a la tabla

pd.set_option("display.max_rows", None, "display.max_columns", None)
data

,N,Sigma,Xi_b,Eta_b,Xib,Etab,Error Xi_b,Error Eta_b
0,0.5000000,0.1000000,2.2899149,2.1743665,2.2899149,2.1743664,0.0000000,0.0000046
1,0.5000000,0.2000000,2.0008463,1.4374281,2.0008463,1.4374279,0.0000000,0.0000139
2,0.5000000,0.3000000,1.8013545,1.0360530,1.8013546,1.0360529,0.0000056,0.0000097
3,0.5000000,0.4000000,1.6543434,0.7912060,1.6543434,0.7912036,0.0000000,0.0003021
4,0.5000000,0.5000000,1.5408662,0.6296190,1.5408662,0.6296189,0.0000000,0.0000222
5,0.5000000,0.6000000,1.4502055,0.5166507,1.4502055,0.5166507,0.0000000,0.0000000
6,0.5000000,0.7000000,1.3758251,0.4341211,1.3758251,0.4341205,0.0000000,0.0001428
7,0.5000000,0.8000000,1.3135016,0.3717053,1.3135016,0.3717034,0.0000000,0.0005085
8,0.5000000,0.9000000,1.2603785,0.3231648,1.2603785,0.3231647,0.0000000,0.0000278
9,1.0000000,0.1000000,2.5990904,1.7514303,2.1040206,2.7351733,19.0478100,56.1679788
